In [1]:
// based on https://github.com/twosigma/beakerx/blob/master/doc/groovy/Tablesaw.ipynb

<console>: 1

In [2]:
%%classpath add mvn
tech.tablesaw tablesaw-beakerx 0.30.3
com.jimmoores quandl-tablesaw 2.0.0
com.github.haifengl smile-core 1.5.2
javax.activation javax.activation-api 1.2.0

In [3]:
%import static tech.tablesaw.aggregate.AggregateFunctions.*
%import tech.tablesaw.api.*
%import tech.tablesaw.columns.*
%import smile.clustering.*
%import smile.regression.*

// display Tablesaw tables with BeakerX table display widget
tech.tablesaw.beakerx.TablesawDisplayer.register()

In [4]:
val tornadoes = Table.read().csv("../resources/data/tornadoes_2014.csv")

In [5]:
//Performing totals and sub-totals
def injuriesByScale = tornadoes.summarize("Injuries", median).by("Scale")
injuriesByScale.setName("Median injuries by Tornado Scale")
injuriesByScale

In [6]:
%import com.jimmoores.quandl.*
%import com.jimmoores.quandl.tablesaw.*

In [7]:
val session:TableSawQuandlSession = TableSawQuandlSession.create();
val table:Table = session.getDataSet(DataSetRequest.Builder.of("WIKI/AAPL").build());
// Create a new column containing the year
val yearColumn:NumberColumn[_] = table.dateColumn("Date").year();
yearColumn.setName("Year");
table.addColumns(yearColumn);
// Create max, min and total volume tables aggregated by year
val summaryMax:Table = table.summarize("Adj. Close", max).by("year");
val summaryMin:Table = table.summarize("Adj. Close", min).by("year");
val summaryVolume:Table = table.summarize("Volume", sum).by("year");
// Create a new table from each of these
val summary = Table.create("Summary", summaryMax.column(0), summaryMax.column(1), 
                       summaryMin.column(1), summaryVolume.column(1));


In [9]:
table.numberColumn("Year").asList()


[2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 201

In [10]:
// Add back a DateColumn to the summary...will be used for plotting
val yearDates:DateColumn = DateColumn.create("YearDate");
for (year <- summary.numberColumn("Year").asDoubleArray()) {
    yearDates.append(java.time.LocalDate.of( year.asInstanceOf[Int], 1, 1));
}
summary.addColumns(yearDates)

summary

In [11]:
import scala.collection.JavaConverters._;
val years = summary.dateColumn("YearDate").asList().asScala;

val plot = new TimePlot();
plot.title= "Price Chart for AAPL";
plot.xLabel="Time";
plot.yLabel="Max [Adj. Close]";
val yAxis=new YAxis();
yAxis.label="Volume";
plot.add(yAxis);
val points=new Points();
points.x=years.toSeq;
points.y=summary.doubleColumn("Max [Adj. Close]").asList.asScala;
plot.add(points)
val line=new Line()
line.x=years
line.y=summary.doubleColumn("Max [Adj. Close]").asList.asScala;
line.color=Color.blue
plot.add(line)
val stems=new Stems()
stems.x=years
stems.y=summary.doubleColumn("Sum [Volume]").asList.asScala;
stems.yAxis="Volume"
plot.add(stems)
plot